<a href="https://colab.research.google.com/github/Welco2004tanvi/ML-Projects/blob/main/Movielens_Recommender_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/movies.csv')


In [5]:
df = pd.read_csv('/content/ratings.csv')

In [8]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [10]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469543 sha256=8183069c9364facbe366dfa0fe7c82761823fbada41aa8cd8b9aa14ab0677db0
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [14]:
 # Uninstall preinstalled NumPy (optional but safer)
!pip uninstall -y numpy

# Reinstall compatible NumPy and surprise library
!pip install numpy==1.23.5
!pip install scikit-surprise


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
db-dtypes 1.

In [2]:
import pandas as pd
from surprise import SVD, KNNBaseline, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split, LeaveOneOut
from collections import defaultdict


In [4]:
!mkdir -p movielens_data
!wget -q --show-progress https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip -d movielens_data


ml-latest-small.zip 100%[===================>] 955.28K  4.29MB/s    in 0.2s    
Archive:  ml-latest-small.zip
   creating: movielens_data/ml-latest-small/
  inflating: movielens_data/ml-latest-small/links.csv  
  inflating: movielens_data/ml-latest-small/tags.csv  
  inflating: movielens_data/ml-latest-small/ratings.csv  
  inflating: movielens_data/ml-latest-small/README.txt  
  inflating: movielens_data/ml-latest-small/movies.csv  


In [6]:
# Create folder and download MovieLens dataset
!mkdir -p movielens_data
!wget -q --show-progress https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip -d movielens_data


ml-latest-small.zip 100%[===================>] 955.28K  4.29MB/s    in 0.2s    
Archive:  ml-latest-small.zip
  inflating: movielens_data/ml-latest-small/links.csv  
  inflating: movielens_data/ml-latest-small/tags.csv  
  inflating: movielens_data/ml-latest-small/ratings.csv  
  inflating: movielens_data/ml-latest-small/README.txt  
  inflating: movielens_data/ml-latest-small/movies.csv  


In [7]:
movies = pd.read_csv('movielens_data/ml-latest-small/movies.csv')
ratings = pd.read_csv('movielens_data/ml-latest-small/ratings.csv')

# Merge the datasets
df = pd.merge(movies, ratings, on='movieId', how='inner')
df.head()


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [8]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)


In [9]:
trainSet, testSet = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(random_state=0)
algo.fit(trainSet)
predictions = algo.test(testSet)


In [10]:
def MAE(predictions):
    return accuracy.mae(predictions, verbose=False)

def RMSE(predictions):
    return accuracy.rmse(predictions, verbose=False)

print("RMSE: ", RMSE(predictions))
print("MAE: ", MAE(predictions))


RMSE:  0.8721506370755425
MAE:  0.6695566522267671


In [11]:
def GetTopN(predictions, n=10, minimumRating=4.0):
    topN = defaultdict(list)
    for userID, movieID, actualRating, estimatedRating, _ in predictions:
        if estimatedRating >= minimumRating:
            topN[int(userID)].append((int(movieID), estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN


In [12]:
LOOCV = LeaveOneOut(n_splits=1, random_state=1)
for trainSet, testSet in LOOCV.split(data):
    algo.fit(trainSet)
    leftOutPredictions = algo.test(testSet)
    bigTestSet = trainSet.build_anti_testset()
    allPredictions = algo.test(bigTestSet)
    topNPredicted = GetTopN(allPredictions, n=10)


In [13]:
def HitRate(topNPredicted, leftOutPredictions):
    hits = 0
    total = 0
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        hit = any(int(leftOutMovieID) == int(movieID) for movieID, _ in topNPredicted[int(userID)])
        if hit: hits += 1
        total += 1
    return hits / total

print("Hit Rate: ", HitRate(topNPredicted, leftOutPredictions))


Hit Rate:  0.02459016393442623


In [14]:
def RatingHitRate(topNPredicted, leftOutPredictions):
    hits = defaultdict(float)
    total = defaultdict(float)
    for userID, leftOutMovieID, actualRating, _, _ in leftOutPredictions:
        hit = any(int(leftOutMovieID) == movieID for movieID, _ in topNPredicted[int(userID)])
        if hit: hits[actualRating] += 1
        total[actualRating] += 1

    for rating in sorted(hits):
        print(f"{rating}: {hits[rating] / total[rating]}")

print("Hit Rate by Rating value: ")
RatingHitRate(topNPredicted, leftOutPredictions)


Hit Rate by Rating value: 
3.0: 0.00909090909090909
4.0: 0.017857142857142856
4.5: 0.038461538461538464
5.0: 0.06766917293233082


In [15]:
def CumulativeHitRate(topNPredicted, leftOutPredictions, ratingCutoff=0):
    hits = 0
    total = 0
    for userID, leftOutMovieID, actualRating, _, _ in leftOutPredictions:
        if actualRating >= ratingCutoff:
            hit = any(int(leftOutMovieID) == movieID for movieID, _ in topNPredicted[int(userID)])
            if hit: hits += 1
            total += 1
    return hits / total

print("Cumulative Hit Rate (rating >= 4): ", CumulativeHitRate(topNPredicted, leftOutPredictions, 4.0))


Cumulative Hit Rate (rating >= 4):  0.039660056657223795


In [16]:
def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
    summation = 0
    total = 0
    for userID, leftOutMovieID, _, _, _ in leftOutPredictions:
        for rank, (movieID, _) in enumerate(topNPredicted[int(userID)], start=1):
            if int(leftOutMovieID) == movieID:
                summation += 1.0 / rank
                break
        total += 1
    return summation / total

print("Average Reciprocal Hit Rank: ", AverageReciprocalHitRank(topNPredicted, leftOutPredictions))


Average Reciprocal Hit Rank:  0.007184491282851939


In [17]:
fullTrainSet = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
simsAlgo = KNNBaseline(sim_options=sim_options)
simsAlgo.fit(fullTrainSet)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
algo.fit(fullTrainSet)
bigTestSet = fullTrainSet.build_anti_testset()
allPredictions = algo.test(bigTestSet)
topNPredicted = GetTopN(allPredictions, n=10)


In [19]:
def UserCoverage(topNPredicted, numUsers, ratingThreshold=0):
    hits = 0
    for userID in topNPredicted:
        if any(predictedRating >= ratingThreshold for _, predictedRating in topNPredicted[userID]):
            hits += 1
    return hits / numUsers

print("User coverage: ", UserCoverage(topNPredicted, fullTrainSet.n_users, ratingThreshold=4.0))


User coverage:  0.9245901639344263


In [21]:
!mkdir -p movielens_data
!wget -q --show-progress https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip -d movielens_data


ml-latest-small.zip 100%[===================>] 955.28K  4.29MB/s    in 0.2s    
Archive:  ml-latest-small.zip
  inflating: movielens_data/ml-latest-small/links.csv  
  inflating: movielens_data/ml-latest-small/tags.csv  
  inflating: movielens_data/ml-latest-small/ratings.csv  
  inflating: movielens_data/ml-latest-small/README.txt  
  inflating: movielens_data/ml-latest-small/movies.csv  


In [24]:
import csv
from collections import defaultdict

def getPopularityRanks():
    ratings = defaultdict(int)
    rankings = defaultdict(int)

    # ✅ Make sure the path is correct
    with open('movielens_data/ml-latest-small/ratings.csv', newline='') as csvfile:
        ratingReader = csv.reader(csvfile)
        next(ratingReader)  # skip header
        for row in ratingReader:
            movieID = int(row[1])
            ratings[movieID] += 1

    # ✅ Create rankings based on popularity
    rank = 1
    for movieID, ratingCount in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
        rankings[movieID] = rank
        rank += 1

    return rankings

# Generate popularity ranks
rankings = getPopularityRanks()

# ✅ Novelty function using popularity rank
def Novelty(topNPredicted, rankings):
    total = 0
    count = 0
    for userID in topNPredicted:
        for movieID, _ in topNPredicted[userID]:
            total += rankings.get(movieID, 0)  # avoid KeyError
            count += 1
    return total / count if count else 0

# ✅ Example usage (you must already have topNPredicted)
# Example dummy structure for demonstration
# topNPredicted = {
#     'user1': [(1, 4.5), (50, 4.3)],
#     'user2': [(10, 4.7), (20, 4.0)]
# }

# Replace this with your real topNPredicted
print("Novelty (average popularity rank):", Novelty(topNPredicted, rankings))


Novelty (average popularity rank): 548.7711769115442
